## Data reading and manipulation

In [1]:
# activate the environment
using Pkg
Pkg.activate("/home/fguney/projects/e2e-drive/WoR/world_on_rails.jl")

  Activating environment at `~/projects/e2e-drive/WoR/world_on_rails.jl/Project.toml`


In [2]:
using YAML: load_file
using Images
using LMDB
using LMDB: create, open, close, put!, start

In [3]:
DATA_ROOT = "/home/fguney/projects/e2e-drive/WoR/dataset"
CONFIG_PATH = "/home/fguney/projects/e2e-drive/WoR/WorldOnRails/config.yaml"

"/home/fguney/projects/e2e-drive/WoR/WorldOnRails/config.yaml"

In [6]:
struct MainDataset
    config::Dict
    num_frames::Int
    txn_map::Dict
    idx_map::Dict
    yaw_map::Dict
    file_map::Dict

    function MainDataset(data_dir::AbstractString, config_path::AbstractString)
        for full_path in readdir(data_dir, join=true)
            if !isdir(full_path)
                continue
            end
            env = create()
            
            open(env, full_path)
            txn = start(env)
            dbi = open(txn)
            
            n = parse(Int64, LMDB.get(txn, dbi, "len", String))
            if n < config["num_plan"]+1
                print(full_path, " is too small. consider deleting")
                close(env, dbi)
            else
                offset = num_frames
                for i in 1:n-config["num_plan"]
                    num_frames+=1
                    for j in 1:length(config["camera_yaws"])
                        txn_map[o]
                    end
                end
            end
        
        end
    end
end



In [7]:
MainDataset(DATA_ROOT, CONFIG_PATH)

full_path = "/home/fguney/projects/e2e-drive/WoR/dataset/adyhsylssx"
n = 200
